# 歡迎來到 **OpenVINO Runtime API(推論引擎)架構與功能介紹** 課程的編程作業!

你將通過完成這份作業, 更加熟悉OpenVINO三步驟快速上手流程, 即:
- 建置 (Build)
- 優化 (Optimize)
- 部署 (Deploy)

首先, 你將在Google Colaboratory (Colab)的環境下安裝[OpenVINO 2022.1 Dev Tools](https://www.intel.com/content/www/us/en/developer/tools/openvino-toolkit/download.html)

接著, 按照所指示的資訊, 你將手把手完成建置, 優化, 以及部署的代碼實現.
- 下載目標影像和標記資料, 以及模型
- 使用[模型優化器](https://docs.openvino.ai/latest/openvino_docs_MO_DG_Deep_Learning_Model_Optimizer_DevGuide.html)將原預訓練模型轉換成[Intermediate Representation (IR)檔案](https://docs.openvino.ai/latest/openvino_docs_MO_DG_IR_and_opsets.html?highlight=intermediate%20representation)
- 利用[OpenVINO Runtime API(推論引擎)](https://docs.openvino.ai/latest/openvino_docs_OV_UG_OV_Runtime_User_Guide.html)做實際的推論與部署

## 0. 前置處理 - 安裝OpenVINO 2022.1 Dev Tools

In [ ]:
!pip install openvino-dev==2022.1.0
!pip install onnx==1.11.0

### Import所需要的函示庫

In [ ]:
import cv2
import os
import numpy as np
from openvino.preprocess import PrePostProcessor, ResizeAlgorithm
from openvino.runtime import Core, Layout, Type
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

## 1.建置


### 1.1下載推論目標影像(一張跑車的照片), 並且將它顯示出來做確認

In [ ]:
!wget --no-check-certificate 'https://drive.google.com/u/0/uc?id=1bINVmgR5Txrg7qvFIo5wm2UWUjRp0rBv&export=download' -O 'car.png'

car = mpimg.imread('car.png')

plt.imshow(car)

### 1.2下載標記資料 (squeezenet1.1 label), 並且將前五個項目印出來做確認

In [ ]:
!wget --no-check-certificate 'https://drive.google.com/u/0/uc?id=1HXV2CPQw5yfJipBBWd0PJNXvmauX_hoI&export=download' -O 'label.txt'

with open('label.txt', 'r') as f:
  label = f.read()

label = label.split('\n')

label[:5]

### 1.3利用OpenVINO的工具 - [Model Downloader](https://docs.openvino.ai/latest/omz_tools_downloader.html), 去下載[mobilenet-v2-pytorch](https://docs.openvino.ai/nightly/omz_models_model_mobilenet_v2_pytorch.html)這個公開的預訓練模型

你將使用`omz_downloader`, openvino-dev tools的命令列指令, 從在線資源下載模型

In [ ]:
base_model_dir = "model"

model_name = "mobilenet-v2-pytorch"

download_cmd = f"omz_downloader " \
               f"--name {model_name} " \
               f"--output_dir {base_model_dir} " \
               f"--cache_dir {base_model_dir}"

!$download_cmd

## 2.優化

## 利用OpenVINO的工具 - [Model Converter](https://docs.openvino.ai/latest/omz_tools_downloader.html), 去轉換下載的mobilenet-v2-pytorch模型成IR格式的檔案

你將使用`omz_converter`, openvino-dev tools的命令列指令, 用模型優化器將不是OpenVINO IR 格式的模型轉換為該格式

In [ ]:
precision = "FP16"

converted_model_path = f"model/public/{model_name}/{precision}/{model_name}.xml"

if not os.path.exists(converted_model_path):
    covert_cmd = f"omz_converter " \
                 f"--name {model_name} " \
                 f"--download_dir {base_model_dir} " \
                 f"--precisions {precision}"
!$covert_cmd

## 3.部署

### 3.1 **[需要完成]** 建立Core物件 (提示, 參考[Inference Pipeline](https://docs.openvino.ai/2022.1/openvino_2_0_inference_pipeline.html)-1: Create Core)

In [ ]:
### your code here ###


### 3.2 **[需要完成]** 讀取模型 (提示, 參考[Inference Pipeline](https://docs.openvino.ai/2022.1/openvino_2_0_inference_pipeline.html)-2: Read a model from a drive¶)

In [ ]:
### your code here ###


### 3.3 利用[Preprocessing API](https://docs.openvino.ai/latest/openvino_docs_OV_UG_Preprocessing_Details.html)做資料前處理

In [ ]:
image = cv2.imread('car.png')
input_tensor = np.expand_dims(image, 0)

ppp = PrePostProcessor(model)
_, h, w, _ = input_tensor.shape
(ppp.input().tensor().set_element_type(Type.u8).set_layout(
    Layout('NHWC')).set_spatial_static_shape(h, w))
ppp.input().preprocess().resize(ResizeAlgorithm.RESIZE_LINEAR)
ppp.input().model().set_layout(Layout('NCHW'))
ppp.output().tensor().set_element_type(Type.f32)
model = ppp.build()

### 3.4 **[需要完成]** 加載模型至所選擇的推論裝置 (提示, 參考[Inference Pipeline](https://docs.openvino.ai/2022.1/openvino_2_0_inference_pipeline.html)-3. Load the Model to the Device¶)

In [ ]:
### your code here ###


### 3.5 **[需要完成]** 建立推論請求, 然後執行推論 (提示, 參考[Inference Pipeline](https://docs.openvino.ai/2022.1/openvino_2_0_inference_pipeline.html)-4. Create an Inference Request, 5. Fill input tensors, 6. Start Inference¶)

In [ ]:
### your code here ###



### 3.6 處理推論結果

In [ ]:
probs = next(iter(results.values()))
idx = np.argsort(probs[0])[::-1]
for i in range(5):
    print(idx[i], round(probs[0][idx[i]], 2), label[idx[i]])

## 請將推論結果儲存, 並返回課程列, 找到可供你上傳assignment-1進行評分的部分. 祝你好運！